# image_handler

In [67]:
import cv2
import numpy as np
from tkinter import filedialog, Toplevel, Canvas, Button, Frame
from PIL import Image, ImageTk, ImageFilter

# Variáveis globais para gerenciar o estado da imagem
imagem_original = None
imagem_atual = None
canvas = None
selection = None  # Variável para armazenar as coordenadas de seleção

# Função para aplicar zoom
def aplicar_zoom(tipo):
    global imagem_atual, canvas

    if imagem_atual is None:
        print("Nenhuma imagem carregada para aplicar zoom.")
        return

    # Definir fator de zoom
    fator = 1.2 if tipo == "mais" else 0.8

    # Calcular novas dimensões
    largura, altura = imagem_atual.size
    nova_largura = int(largura * fator)
    nova_altura = int(altura * fator)

    # Redimensionar imagem
    imagem_atual = imagem_atual.resize((nova_largura, nova_altura), Image.Resampling.LANCZOS)
    imagem_tk = ImageTk.PhotoImage(imagem_atual)

    # Atualizar imagem no Canvas
    canvas.img = imagem_tk  # Evita garbage collection
    canvas.delete("all")
    canvas.create_image(0, 0, anchor="nw", image=imagem_tk)

# Função para aplicar o filtro
def aplicar_filtro(filtro):
    global imagem_atual, canvas

    if imagem_atual is None:
        print("Nenhuma imagem carregada para aplicar o filtro.")
        return

    if filtro == "blur":
        imagem_atual = imagem_atual.filter(ImageFilter.BLUR)
    elif filtro == "sharpen":
        imagem_atual = imagem_atual.filter(ImageFilter.SHARPEN)
    elif filtro == "emboss":
        imagem_atual = imagem_atual.filter(ImageFilter.EMBOSS)
    elif filtro == "laplacian":
        imagem_atual = aplicar_filtro_laplacian(imagem_atual)
    elif filtro == "canny":
        imagem_atual = aplicar_filtro_canny(imagem_atual)
    elif filtro == "sobel":
        imagem_atual = aplicar_filtro_sobel(imagem_atual)

    # Exibir a imagem filtrada no canvas
    imagem_tk = ImageTk.PhotoImage(imagem_atual)
    canvas.img = imagem_tk  # Evita garbage collection
    canvas.delete("all")
    canvas.create_image(0, 0, anchor="nw", image=imagem_tk)

# Funções para filtros avançados (Laplacian, Canny, Sobel)
def aplicar_filtro_laplacian(imagem):
    imagem_cv = np.array(imagem.convert('L'))  # Converte para escala de cinza
    laplacian = cv2.Laplacian(imagem_cv, cv2.CV_64F)
    laplacian = cv2.convertScaleAbs(laplacian)
    return Image.fromarray(laplacian)

def aplicar_filtro_canny(imagem):
    imagem_cv = np.array(imagem.convert('L'))
    canny = cv2.Canny(imagem_cv, 100, 200)  # Limites para o Canny
    return Image.fromarray(canny)

def aplicar_filtro_sobel(imagem):
    imagem_cv = np.array(imagem.convert('L'))
    sobelx = cv2.Sobel(imagem_cv, cv2.CV_64F, 1, 0, ksize=3)  # Filtro Sobel no eixo X
    sobely = cv2.Sobel(imagem_cv, cv2.CV_64F, 0, 1, ksize=3)  # Filtro Sobel no eixo Y
    sobel = cv2.magnitude(sobelx, sobely)
    sobel = np.uint8(np.abs(sobel))  # Absoluto para garantir valores positivos
    return Image.fromarray(sobel)

# Função para voltar para a imagem original
def voltar_para_original():
    global imagem_atual, imagem_original, canvas

    if imagem_original is None:
        print("Nenhuma imagem original carregada para restaurar.")
        return

    imagem_atual = imagem_original.copy()  # Restaurar a imagem original
    imagem_tk = ImageTk.PhotoImage(imagem_atual)

    canvas.img = imagem_tk  # Evita garbage collection
    canvas.delete("all")
    canvas.create_image(0, 0, anchor="nw", image=imagem_tk)

# Função para selecionar e recortar a imagem
def selecionar_recortar():
    global canvas, imagem_atual, selection

    if imagem_atual is None:
        print("Nenhuma imagem carregada para selecionar e recortar.")
        return

    # Habilitar o modo de seleção
    canvas.bind("<ButtonPress-1>", iniciar_selecao)
    canvas.bind("<B1-Motion>", fazer_selecao)
    canvas.bind("<ButtonRelease-1>", finalizar_selecao)

# Funções para manipulação de seleção de área
def iniciar_selecao(event):
    global selection
    selection = [event.x, event.y]  # Armazenar posição inicial da seleção

def fazer_selecao(event):
    global canvas, selection
    if selection:
        canvas.delete("selection")
        canvas.create_rectangle(
            selection[0], selection[1], event.x, event.y, outline="red", tags="selection"
        )

def finalizar_selecao(event):
    global canvas, selection, imagem_atual

    if selection:
        x1, y1 = selection
        x2, y2 = event.x, event.y

        # Certificar-se de que as coordenadas estão corretas
        left = min(x1, x2)
        top = min(y1, y2)
        right = max(x1, x2)
        bottom = max(y1, y2)

        # Recortar a imagem
        imagem_recortada = imagem_atual.crop((left, top, right, bottom))

        # Atualizar a imagem atual com o recorte
        imagem_atual = imagem_recortada

        # Exibir a imagem recortada
        imagem_tk = ImageTk.PhotoImage(imagem_recortada)
        canvas.img = imagem_tk  # Evita garbage collection
        canvas.delete("all")
        canvas.create_image(0, 0, anchor="nw", image=imagem_tk)

        # Limpar a seleção
        canvas.delete("selection")
        selection = None


# Funções de placeholder
def converter_imagem(tipo):
    print(f"Imagem convertida para {tipo}.")

# Função para abrir e exibir a imagem
def abrir_imagem():
    caminho = filedialog.askopenfilename(filetypes=[("Imagens", "*.png;*.jpg;*.jpeg;*.bmp")])
    if caminho:
        exibir_imagem(caminho)

# Função para exibir a imagem no Canvas
def exibir_imagem(caminho):
    global imagem_atual, imagem_original, canvas  # Usando as variáveis globais

    # Nova janela para exibir a imagem
    janela_imagem = Toplevel()
    janela_imagem.title("Editar Imagem")

    # Configurar o frame principal
    frame = Frame(janela_imagem)
    frame.pack()

    # Canvas para a imagem
    canvas = Canvas(frame, width=500, height=500, bg="gray")
    canvas.grid(row=0, column=0, columnspan=3)

    # Abrir e redimensionar a imagem
    imagem_atual = Image.open(caminho)  # Agora a imagem_atual é global
    imagem_original = imagem_atual.copy()  # Armazenar a imagem original
    imagem_atual.thumbnail((500, 500))
    imagem_tk = ImageTk.PhotoImage(imagem_atual)

    canvas.img = imagem_tk  # Evita garbage collection
    canvas.create_image(0, 0, anchor="nw", image=imagem_tk)

    # Adicionar botões para funcionalidades
    Button(frame, text="Zoom +", command=lambda: aplicar_zoom("mais")).grid(row=1, column=0, pady=10)
    Button(frame, text="Zoom -", command=lambda: aplicar_zoom("menos")).grid(row=1, column=1, pady=10)
    Button(frame, text="Filtro Blur", command=lambda: aplicar_filtro("blur")).grid(row=1, column=2, pady=10)
    Button(frame, text="Filtro Sharpen", command=lambda: aplicar_filtro("sharpen")).grid(row=1, column=3, pady=10)
    Button(frame, text="Filtro Emboss", command=lambda: aplicar_filtro("emboss")).grid(row=1, column=4, pady=10)
    Button(frame, text="Filtro Laplacian", command=lambda: aplicar_filtro("laplacian")).grid(row=2, column=0, pady=10)
    Button(frame, text="Filtro Canny", command=lambda: aplicar_filtro("canny")).grid(row=2, column=1, pady=10)
    Button(frame, text="Filtro Sobel", command=lambda: aplicar_filtro("sobel")).grid(row=2, column=2, pady=10)
    Button(frame, text="Voltar ao Original", command=voltar_para_original).grid(row=3, column=0, pady=10)
    Button(frame, text="Selecionar e Recortar", command=selecionar_recortar).grid(row=3, column=1, pady=10)
    Button(frame, text="Converter (Cinza)", command=lambda: converter_imagem("tons de cinza")).grid(row=3, column=3, pady=10)

# video_handler

In [68]:
from tkinter import Toplevel, Label

def abrir_video():
    # Placeholder para funcionalidade de vídeo
    janela_video = Toplevel()
    janela_video.title("Visualizar Vídeo")
    Label(janela_video, text="Funcionalidade de vídeo em desenvolvimento!").pack()

# main_application

In [69]:
import tkinter as tk

def main():
    # Tela principal
    janela_principal = tk.Tk()
    janela_principal.title("Editor de Imagens e Vídeos")

    # Texto de introdução
    label = tk.Label(janela_principal, text="Escolha o que deseja editar:", font=("Arial", 14))
    label.pack(pady=20)

    # Botões para escolher Imagem ou Vídeo
    botao_imagem = tk.Button(janela_principal, text="Editar Imagem", command=abrir_imagem, width=20)
    botao_imagem.pack(pady=10)

    botao_video = tk.Button(janela_principal, text="Editar Vídeo", command=abrir_video, width=20)
    botao_video.pack(pady=10)

    # Inicia a interface
    janela_principal.mainloop()

main()


Imagem convertida para tons de cinza.
